In [1]:
import numpy as np

In [2]:
class RangeFilter:
    """
    A RangeFilter class that filters lidar scan data based on a minimum and maximum range

    ...

    Attributes
    ----------
    range_min : float
        minimum range value
        all values less than range_min will be cropped to range_min.
    range_max : float
        maximum range value
        all values greater than range_max will be cropped to range_max.

    """
    
    def __init__(self, range_min, range_max):
        """
        Initialize RangeFilter object
        
        Args:
            range_min: minimum range value, all values less than range_min will be cropped to range_min.
            range_max: maximum range value, all values greater than range_max will be cropped to range_max.

        """
        # Store member variables
        self.range_min = range_min
        self.range_max = range_max

    def update(self, lidar_scan):
        """
        Filter lidar_scan using a range filter
        
        Args:
            lidar_scan: numpy array of lidar range data
            
        Returns:
            a numpy array with the same length as lidar_scan, which has been range filtered
            all values in lidar_scan less than range_min will be changed to range_min
            all values in lidar_scan greater than range_max will be changed to range_max            

        """
        # Make copy to ensure no changes made to input 
        filtered_lidar_scan = np.copy(lidar_scan)

        # Perform range filtering
        filtered_lidar_scan[filtered_lidar_scan < self.range_min] = self.range_min
        filtered_lidar_scan[filtered_lidar_scan > self.range_max] = self.range_max
        return filtered_lidar_scan

In [3]:
class TemporalMedianFilter:
    """
    A TemporalMedianFilter class that filters lidar scan data based on a median of previous scans
    ...

    Attributes
    ----------
    D : int
        number of previous scans to look at for median calculation (must be a non-negative integer) 
        finds median of current and previous D scans (total of D+1 scans)
    lidar_scans : list
        list of previous lidar scans to perform median calculation

    """
    
    def __init__(self, D):
        """
        Initialize TemporalMedianFilter object
        
        Args:
            D: number of previous scans to look at for median calculation (must be a non-negative integer) 

        """
        # Initialize member variables
        self.D = D
        self.lidar_scans = []

    def update(self, lidar_scan):
        """
        Filter lidar_scan using a temporal median filter
        
        Args:
            lidar_scan: numpy array of lidar range data
            
        Returns:
            a numpy array with the same length as lidar_scan, after passing through a temporal median filter
            each element in the returned numpy array is the median of the current and previous D scans
            for the first D scans, this will return the median of all the scans so far
            y_i(t) = median(x_i(t) + x_i(t-1) + x_i(t-2) ... + x_i(t-D))

        """
        # Remove stale scans
        if (len(self.lidar_scans) > self.D):
            self.lidar_scans = self.lidar_scans[-self.D:]
            
        # Add new lidar scans and compute median
        self.lidar_scans.append(np.copy(lidar_scan))
        np_lidar_scans = np.array(self.lidar_scans)
        return np.median(np_lidar_scans, axis=0)     

In [4]:
# Parameters
range_min = 5
range_max = 20
D = 3
N = 5

data_min = 0.03
data_spread = 49.97

In [5]:
# Create filter objects
rf = RangeFilter(range_min, range_max)
tmf = TemporalMedianFilter(D)

In [6]:
# Test lidar data
ls1 = np.random.rand(N) * data_spread + data_min
ls2 = np.random.rand(N) * data_spread + data_min
ls3 = np.random.rand(N) * data_spread + data_min
ls4 = np.random.rand(N) * data_spread + data_min
ls5 = np.random.rand(N) * data_spread + data_min
ls6 = np.random.rand(N) * data_spread + data_min
ls7 = np.random.rand(N) * data_spread + data_min
ls8 = np.random.rand(N) * data_spread + data_min

In [7]:
# Show lidar data
print((ls1))
print((ls2))
print((ls3))
print((ls4))
print((ls5))
print((ls6))

[29.29051424 44.48822432 15.29978941 38.81336441 45.38223664]
[39.68742477 40.99351242 10.12709469 31.40417462  1.8844951 ]
[44.39067702 19.65880271 36.47119016 15.85818929 38.81647709]
[10.5227827  46.34054304 26.49107115 12.76173665 15.03798254]
[27.94868514 22.35665449 19.38224579 27.53125167 22.97806083]
[ 4.71423981 36.7038233   1.53809036  8.70066262  2.14240958]


In [8]:
# Test RangeFilter
print(rf.update(ls1))
print(rf.update(ls2))
print(rf.update(ls3))
print(rf.update(ls4))
print(rf.update(ls5))
print(rf.update(ls6))

[20.         20.         15.29978941 20.         20.        ]
[20.         20.         10.12709469 20.          5.        ]
[20.         19.65880271 20.         15.85818929 20.        ]
[10.5227827  20.         20.         12.76173665 15.03798254]
[20.         20.         19.38224579 20.         20.        ]
[ 5.         20.          5.          8.70066262  5.        ]


In [9]:
# Test Temporal Median Filter
print(tmf.update(ls1))
print(tmf.update(ls2))
print(tmf.update(ls3))
print(tmf.update(ls4))
print(tmf.update(ls5))
print(tmf.update(ls6))

[29.29051424 44.48822432 15.29978941 38.81336441 45.38223664]
[34.4889695  42.74086837 12.71344205 35.10876952 23.63336587]
[39.68742477 40.99351242 15.29978941 31.40417462 38.81647709]
[34.4889695  42.74086837 20.89543028 23.63118196 26.92722982]
[33.81805495 31.67508346 22.93665847 21.69472048 19.00802168]
[19.23573392 29.5302389  22.93665847 14.30996297 19.00802168]


In [10]:
print(type(1))

<class 'int'>


In [11]:
print(type(tmf.lidar_scans))

<class 'list'>
